In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.display import display,HTML
from datetime import datetime

pd.set_option('display.max_colwidth', None)

def path_to_image_html(path):
    return '<img src="'+ path + '" width="120" >'

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

/tmp/ipykernel_6865/3304553925.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
page = requests.get("https://bananafingers.co.uk/outlet", headers = headers)
assert page.status_code == 200

In [3]:
soup = bs(page.text, 'html.parser')
sale = int(soup.findAll('span', class_='toolbar-number')[2].get_text())
pgs = sale/12 if (sale/12)%1 == 0 else int(sale/12)+1
print(f'pages to scan : {pgs}')

In [6]:
dict = {'item_name':[],
        'price':[],
        'pecentage_off':[],
        'previous_price':[],
        'img_link':[],
        'item_link':[]}

for pg in range(1, pgs + 1):
    url = f'https://bananafingers.co.uk/outlet?p={pg}'
    page = requests.get(url, headers = headers)
    assert page.status_code == 200
    soup = bs(page.text, 'html.parser')
    all = soup.findAll('div', class_ = 'product-item-info')

    for i in all:
        pecentage_off = float(i.find('div', class_ = 'product-labels').get_text().strip().replace('%', ''))
        dict['pecentage_off'].append(pecentage_off)
        dict['img_link'].append( i.find('img')['src'])
        dict['item_name'].append(i.find(class_='product-item-link').get_text().strip())
        dict['item_link'].append( i.find(class_='product-item-link')['href'])
        price = float(i.find(class_='price').get_text().strip('£'))
        dict['price'].append(price)
        dict['previous_price'].append(price / (1-(pecentage_off/100))
    
    print(f'page {pg} finished at {datetime.now().strftime("%H:%M:%S")}')

new = pd.DataFrame(dict)
new.sort_values('pecentage_off', inplace =True, ascending = False)
new.reset_index(drop=True, inplace = True)

page 1 finished at 05:56:08
page 2 finished at 05:56:09
page 3 finished at 05:56:09
page 4 finished at 05:56:10
page 5 finished at 05:56:10
page 6 finished at 05:56:10
page 7 finished at 05:56:11
page 8 finished at 05:56:11
page 9 finished at 05:56:12
page 10 finished at 05:56:23
page 11 finished at 05:56:24
page 12 finished at 05:56:24
page 13 finished at 05:56:25
page 14 finished at 05:56:25
page 15 finished at 05:56:25
page 16 finished at 05:56:26
page 17 finished at 05:56:26
page 18 finished at 05:56:27
page 19 finished at 05:56:27
page 20 finished at 05:56:28
page 21 finished at 05:56:28
page 22 finished at 05:56:28
page 23 finished at 05:56:29
page 24 finished at 05:56:29
page 25 finished at 05:56:30
page 26 finished at 05:56:30
page 27 finished at 05:56:31
page 28 finished at 05:56:31
page 29 finished at 05:56:31
page 30 finished at 05:56:32
page 31 finished at 05:56:32
page 32 finished at 05:56:33
page 33 finished at 05:56:33
page 34 finished at 05:56:59
page 35 finished at 05:

In [21]:
new = pd.DataFrame(dict)
new.sort_values('pecentage_off', inplace =True, ascending = False)
new.reset_index(drop=True, inplace = True)
new['previous_price'] = new['previous_price'].apply(lambda x: "£{:.2f}".format((x)))
new['price'] = new['price'].apply(lambda x: "£{:.2f}".format((x)))
new['pecentage_off'] = new['pecentage_off'].apply(lambda x: "{:.0f}%".format((x*100)))

In [24]:
format_dict = {'img_link':path_to_image_html, 'item_link':make_clickable}

In [29]:
show = min(50, len(new))
display(HTML(new.head(show).to_html(escape=False, formatters=format_dict, index = False)))

item_name,price,pecentage_off,previous_price,img_link,item_link
Evolv Zenist Women's,£49.00,67%,£148.48,,https://bananafingers.co.uk/evolv-zenist-women-s
Evolv Geshido,£55.00,66%,£161.76,,https://bananafingers.co.uk/evolv-geshido
Evolv Geshido Women's,£55.00,66%,£161.76,,https://bananafingers.co.uk/evolv-geshido-women-s
EB Strange,£50.00,62%,£131.58,,https://bananafingers.co.uk/eb-strange
Grivel Mountain Runner EVO 10,£46.04,54%,£100.09,,https://bananafingers.co.uk/grivel-mountain-runner-evo-10
Adidas Terrex Speed Flow - Womens,£45.50,54%,£98.91,,https://bananafingers.co.uk/adidas-terrex-speed-flow-womens
Adidas Terrex Speed Flow - Womens,£45.50,54%,£98.91,,https://bananafingers.co.uk/adidas-terrex-speed-flow-womens
Black Diamond Men's Forged Denim Pants,£46.07,54%,£100.15,,https://bananafingers.co.uk/black-diamond-forged-denim-pants
8BPlus Arja Backpack,£39.50,54%,£85.87,,https://bananafingers.co.uk/8bplus-arja-backpack
Evolv Geshido Lace Women's,£75.00,53%,£159.57,,https://bananafingers.co.uk/evolv-geshido-lace-women-s
